In [1]:
!pip install xin-util

In [2]:
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph
from model_trainingtime_prediction.model_level_utils_cnn import convert_cnn2d_data

In [3]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_start_time = time.time()
        self.epoch_times = []
        self.batch_times = []
        self.epoch_times_detail = []
        self.batch_times_detail = []

    def on_train_end(self, logs={}):
        self.train_end_time = time.time()

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        epoch_time_end = time.time()
        self.epoch_times.append(epoch_time_end - self.epoch_time_start)
        self.epoch_times_detail.append((self.epoch_time_start, epoch_time_end))

    def on_train_batch_begin(self, batch, logs={}):
        self.bacth_time_start = time.time()

    def on_train_batch_end(self, batch, logs={}):
        batch_time_end = time.time()
        self.batch_times.append(batch_time_end - self.bacth_time_start)
        self.batch_times_detail.append((self.bacth_time_start, batch_time_end))

    def relative_by_train_start(self):
        self.epoch_times_detail = np.array(self.epoch_times_detail) - self.train_start_time
        self.batch_times_detail = np.array(self.batch_times_detail) - self.train_start_time
        self.train_end_time = np.array(self.train_end_time) - self.train_start_time


In [4]:
classes = 1000

In [5]:
Models = []
Names = []
input_shapes = []

In [6]:
Xception = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(299, 299, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(Xception)
Names.append('Xception')
input_shapes.append((299, 299, 3))

2022-05-15 13:16:54.223667: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
VGG16 = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(VGG16)
Names.append('VGG16')
input_shapes.append((224, 224, 3))

In [8]:
VGG19 = tf.keras.applications.VGG19(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(VGG19)
Names.append('VGG19')
input_shapes.append((224, 224, 3))

In [9]:
ResNet50 = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(ResNet50)
Names.append('ResNet50')
input_shapes.append((224, 224, 3))

In [10]:
ResNet50V2 = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(ResNet50V2)
Names.append('ResNet50V2')
input_shapes.append((224, 224, 3))

In [11]:
ResNet101 = tf.keras.applications.ResNet101(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",

)

Models.append(ResNet101)
Names.append('ResNet101')
input_shapes.append((224, 224, 3))

In [12]:
ResNet152 = tf.keras.applications.ResNet152(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(ResNet152)
Names.append('ResNet152')
input_shapes.append((224, 224, 3))

In [13]:
MobileNet = tf.keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(MobileNet)
Names.append('MobileNet')
input_shapes.append((224, 224, 3))

In [14]:
DenseNet121 = tf.keras.applications.DenseNet121(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
)

Models.append(DenseNet121)
Names.append('DenseNet121')
input_shapes.append((224, 224, 3))

In [15]:
EfficientNetB0 = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
    classifier_activation="softmax",
)

Models.append(EfficientNetB0)
Names.append('EfficientNetB0')
input_shapes.append((224, 224, 3))

In [16]:
DenseNet121 = tf.keras.applications.DenseNet121(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='max',
    classes=classes,
)

Models.append(DenseNet121)
Names.append('DenseNet121')
input_shapes.append((224, 224, 3))

In [17]:
trials = 2
truncate_from = 1
epochs = 10
batch_size = 32
batch_per_epoch = 10

In [18]:
batch_times = []
epoch_times = []
setup_times = []

In [19]:
def get_time(x,y,batch_size, epochs, model, trials):
    batch_size_data_batch = []
    batch_size_data_epoch = []
    for _ in range(trials):
        time_callback = TimeHistory()
        model.fit(
            x,
            y,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[time_callback],
            verbose=False
        )
        times_batch = np.array(time_callback.batch_times) * 1000
        times_epoch = np.array(time_callback.epoch_times) * 1000
        batch_size_data_batch.extend(times_batch)
        batch_size_data_epoch.extend(times_epoch)

    batch_times_truncated = batch_size_data_batch[truncate_from:]
    epoch_times_trancuted = batch_size_data_epoch[truncate_from:]
    recovered_time = [
        np.median(batch_times_truncated)
    ] * truncate_from + batch_times_truncated


    batch_time = np.median(batch_times_truncated)
    epoch_time = np.median(epoch_times_trancuted)
    setup_time = np.sum(batch_size_data_batch) - sum(recovered_time)
    return batch_time, epoch_time, setup_time

In [20]:
for idx, model in enumerate(tqdm(Models)):
    name = Names[idx]
    input_shape = input_shapes[idx]
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
    x = np.ones((batch_size*batch_per_epoch, *input_shape), dtype=np.float32)
    y = np.ones((batch_size*batch_per_epoch, classes), dtype=np.float32)
    batch_time, epoch_time, setup_time = get_time(x,y,batch_size, epochs, model, trials)
    batch_times.append(batch_time)
    epoch_times.append(epoch_time)
    setup_times.append(setup_time)


  9%|███▋                                    | 1/11 [56:06<9:21:02, 3366.22s/it]


KeyboardInterrupt: 

In [21]:
Flops = [convert_cnn2d_data.get_flops_conv2d_keras(input_shapes[idx], conv_model_obj, sum_all=True, add_pooling=True) for idx, conv_model_obj in enumerate(Models)]

In [22]:
Flops1 = [convert_cnn2d_data.get_flops_tensorflow_graph(conv_model_obj) for idx, conv_model_obj in enumerate(Models)]

2022-05-15 14:14:28.078664: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:14:28.078772: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:14:28.086914: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.005ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated

2022-05-15 14:14:30.175894: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:14:30.175958: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:14:30.177738: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:14:43.590970: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:14:43.591064: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:14:43.594251: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



...


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:15:24.051920: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:15:24.052003: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:15:24.064588: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:15:26.269327: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:15:26.269389: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:15:26.284687: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

In [23]:


Flops2 = [convert_cnn2d_data.get_flops_tensorflow_graph2(conv_model_obj) for idx, conv_model_obj in enumerate(Models)]


2022-05-15 14:15:28.427154: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:15:28.427215: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:15:28.434254: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:15:30.956021: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:15:30.956080: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:15:30.957996: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:15:45.037499: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:15:45.037572: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:15:45.039533: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.



...


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:16:26.163018: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:16:26.163081: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:16:26.175079: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

2022-05-15 14:16:28.437481: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-05-15 14:16:28.437545: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-05-15 14:16:28.453933: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.




=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. Note: Please read the implementation for th

In [24]:
data_shapes = [sum([np.prod(i) for i in convert_cnn2d_data.get_data_shape_flow_conv2d_keras(input_shapes[idx], conv_model_obj, start_from=1, up_to=3, conv_weight=1, pool_weight=1)[0]])
               for idx, conv_model_obj in enumerate(Models)]

In [ ]:
plt.figure(figsize=(15, 8))
plt.scatter(Flops1, batch_times)
for x,y,s in zip(Flops1, batch_times, Names):
    plt.text(x,y,s)

In [ ]:
plt.figure(figsize=(15, 8))
plt.scatter(Flops1, setup_times)
for x,y,s in zip(Flops1, setup_times, Names):
    plt.text(x,y,s)